In [10]:
%cd loqui
! git checkout prepare-all-videos
! git pull origin test-model-single-video

!pip install -r requirements.txt

[Errno 2] No such file or directory: 'loqui-refactor'
/tf
fatal: not a git repository (or any parent up to mount point /)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
fatal: not a git repository (or any parent up to mount point /)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [11]:
!cat requirements.txt

cat: requirements.txt: No such file or directory


In [12]:
import torch
import cv2
import numpy as np
from utils.augmenter import center_crop

ModuleNotFoundError: No module named 'utils'

In [ ]:
import cv2
import numpy as np
import torch

def preprocess_frames(frames):
    processed_frames = []

    for frame in frames:
        # Decode the frame data
        img = cv2.imdecode(np.frombuffer(frame, np.uint8), -1)

        # Resize the frame
        resized_frame = cv2.resize(img, input_shape)

        # Convert frame to grayscale
        grayscale_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)

        # Normalize the frame
        normalized_frame = grayscale_frame.astype(np.float32) / 255.0

        # Add frame to the list of processed frames
        processed_frames.append(normalized_frame)

    # Stack frames to create a tensor with shape [num_frames, height, width]
    tensor_frames = np.stack(processed_frames)

    # Add a channel dimension to the tensor
    tensor_frames = np.expand_dims(tensor_frames, axis=1)

    # Convert frames to tensor
    tensor_frames = torch.tensor(tensor_frames)

    return tensor_frames


In [ ]:

jpeg = TurboJPEG()

def extract_opencv(file_name: str) -> list:
    """
    Gets a path to a video file and tries to extract the ROI from it.
    :param file_name: Path to the video file.
    :return: ROI of the given video file.
    """

    video = []
    cap = cv2.VideoCapture(file_name)

    while cap.isOpened():
        ret, frame = cap.read()  # BGR
        if ret:
            roi = frame[115:211, 79:175]
            video.append(roi)
        else:
            break
    cap.release()

    return video

In [ ]:
def plot_frames(frames_to_plot):
    num_frames = frames_to_plot.shape[0]
    for i in range(num_frames):
        frame = numpy_frames[i]  # Extract the frame
        if frame.ndim == 3:  # If the frame is 3D, reshape it to 2D
            frame = frame.squeeze()
        plt.imshow(frame, cmap='gray')
        plt.axis('off')
        plt.show()

In [ ]:
%cd learn-an-effective-lip-reading-model-without-pains
!git pull origin master

In [ ]:
def load_missing(model, pretrained_dict):
    model_dict = model.state_dict()
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict.keys() and v.size() == model_dict[k].size()}                
    missed_params = [k for k, v in model_dict.items() if not k in pretrained_dict.keys()]
    
    print('loaded params/tot params:{}/{}'.format(len(pretrained_dict),len(model_dict)))
    print('miss matched params:',missed_params)
    model_dict.update(pretrained_dict)
    model.load_state_dict(model_dict)
    return model

In [ ]:
class args_to_send():
    n_class = 5
    se = False
    border = False


In [ ]:
%matplotlib inline
!ls
!pwd

In [ ]:
import cv2
from model.model import VideoModel
import torchvision.transforms as transforms
from turbojpeg import TurboJPEG
import matplotlib.pyplot as plt


# Define the desired input shape for the video model
input_shape = (88, 88)  # Adjust the dimensions according to the model's requirements


video_model = VideoModel(args_to_send)
weight = torch.load("/tf/loqui/checkpoints/lrw-aviad/_iter_19559_epoch_3_v_acc_0.52400_.pt", map_location=torch.device('cpu'))
load_missing(video_model, weight.get('video_model'))
video_model.eval()

# Replace 'filename.mp4' with the path to your video file
filename = '/tf/single-videos/ACTION_01002.mp4'

# Get the video frames
frames = extract_opencv(filename)

# Preprocess the frames
frames = preprocess_frames(frames)

# Access the preprocessed frames tensor
tensor_frames = frames.squeeze(0)

# Convert the tensor frames to numpy array
numpy_frames = tensor_frames.numpy()

plot_frames(numpy_frames)


# Pass the frames through the model
with torch.no_grad():
    frames = frames.unsqueeze(0)
    predictions = video_model(frames)
    print(f"predictions: {predictions}")
# Get the predicted label
predicted_label = torch.argmax(predictions)
print(predicted_label)

predicted_label = predicted_label.item()

# Print the predicted label
print(f'Predicted label: {predicted_label}')

In [ ]:
(1000, 1)